In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [59]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [60]:
df = fetch_hourly_rides(12)

2025-03-05 04:16:33,710 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:16:33,719 INFO: Initializing external client
2025-03-05 04:16:33,719 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:16:34,209 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.20s) 


In [61]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-04 16:00:00+00:00,83,0
1,2025-03-05 04:00:00+00:00,90,6
2,2025-03-04 18:00:00+00:00,68,218
3,2025-03-04 23:00:00+00:00,209,4
4,2025-03-04 22:00:00+00:00,211,54
...,...,...,...
4279,2025-03-04 23:00:00+00:00,146,5
4280,2025-03-05 06:00:00+00:00,168,4
4281,2025-03-04 16:00:00+00:00,190,0
4282,2025-03-05 03:00:00+00:00,46,0


In [62]:
df_pred = fetch_predictions(12)

2025-03-05 04:16:45,164 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 04:16:45,172 INFO: Initializing external client


2025-03-05 04:16:45,173 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 04:16:45,821 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214710
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (14.47s) 


In [63]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,13,31.0,2025-03-05 08:00:00+00:00
1,41,15.0,2025-03-05 08:00:00+00:00
2,53,0.0,2025-03-05 08:00:00+00:00
3,81,0.0,2025-03-05 08:00:00+00:00
4,93,0.0,2025-03-05 08:00:00+00:00
...,...,...,...
751,144,10.0,2025-03-05 10:00:00+00:00
752,178,0.0,2025-03-05 10:00:00+00:00
753,29,0.0,2025-03-05 10:00:00+00:00
754,43,88.0,2025-03-05 10:00:00+00:00


In [64]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [65]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-05 08:00:00+00:00,215,3,0.0
1,2025-03-05 08:00:00+00:00,224,14,3.0
2,2025-03-05 08:00:00+00:00,198,0,0.0
3,2025-03-05 08:00:00+00:00,18,2,0.0
4,2025-03-05 08:00:00+00:00,189,1,0.0
...,...,...,...,...
247,2025-03-05 08:00:00+00:00,49,4,1.0
248,2025-03-05 08:00:00+00:00,43,87,99.0
249,2025-03-05 08:00:00+00:00,230,130,106.0
250,2025-03-05 08:00:00+00:00,75,48,18.0


In [66]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [67]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
36,2025-03-05 08:00:00+00:00,2,0,0.0,0.0
126,2025-03-05 08:00:00+00:00,3,0,0.0,0.0
117,2025-03-05 08:00:00+00:00,4,15,20.0,5.0
201,2025-03-05 08:00:00+00:00,6,0,0.0,0.0
241,2025-03-05 08:00:00+00:00,7,7,2.0,-5.0
...,...,...,...,...,...
159,2025-03-05 08:00:00+00:00,259,0,0.0,0.0
90,2025-03-05 08:00:00+00:00,260,3,1.0,-2.0
138,2025-03-05 08:00:00+00:00,261,15,8.0,-7.0
191,2025-03-05 08:00:00+00:00,262,196,41.0,-155.0


In [68]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-05 08:00:00+00:00,215,3,0.0,-3.0
1,2025-03-05 08:00:00+00:00,224,14,3.0,-11.0
2,2025-03-05 08:00:00+00:00,198,0,0.0,0.0
3,2025-03-05 08:00:00+00:00,18,2,0.0,-2.0
4,2025-03-05 08:00:00+00:00,189,1,0.0,-1.0
...,...,...,...,...,...
247,2025-03-05 08:00:00+00:00,49,4,1.0,-3.0
248,2025-03-05 08:00:00+00:00,43,87,99.0,12.0
249,2025-03-05 08:00:00+00:00,230,130,106.0,-24.0
250,2025-03-05 08:00:00+00:00,75,48,18.0,-30.0


In [69]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [70]:
mae_by_hour["MAE"].mean()

10.869047619047619